In [21]:
import openai
import io
import pandas as pd

## OpenAI Vector Store

In [2]:
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [10]:
client = OpenAI()


In [11]:
vector_store = client.vector_stores.create(
    name="hot_links"
)
print(vector_store.id)

vs_68ac4cff5f1c81919b3bfda45ea803bb


Based on [OpenAI documentation](https://platform.openai.com/docs/guides/tools-file-search#how-to-use)

In [14]:
b = io.BytesIO(b"some initial text data")

In [16]:
b.getvalue()

b'some initial text data'

In [22]:
from openai.types import FileObject

In [24]:
?FileObject

Init signature:
FileObject(
    *,
    id: str,
    bytes: int,
    created_at: int,
    filename: str,
    object: Literal['file'],
    purpose: Literal['assistants', 'assistants_output', 'batch', 'batch_output', 'fine-tune', 'fine-tune-results', 'vision', 'user_data'],
    status: Literal['uploaded', 'processed', 'error'],
    expires_at: Optional[int] = None,
    status_details: Optional[str] = None,
    **extra_data: Any,
) -> None
Docstring:     
!!! abstract "Usage Documentation"
    [Models](../concepts/models.md)

A base class for creating Pydantic models.

Attributes:
    __class_vars__: The names of the class variables defined on the model.
    __private_attributes__: Metadata about the private attributes of the model.
    __signature__: The synthesized `__init__` [`Signature`][inspect.Signature] of the model.

    __pydantic_complete__: Whether model building is completed, or if there are still undefined fields.
    __pydantic_core_schema__: The core schema of the model.
   

In [45]:
io.BytesIO('test'.encode('utf-8'))

In [82]:
def make_file_object(name,content):
    return (name+'.txt', io.BytesIO(content.encode('utf-8')), 'text/markdown')
#

In [27]:
def add_file_object(file_object):
    client.vector_stores.files.upload_and_poll(        # Upload file
    vector_store_id=vector_store.id,
    file=file_object   
)

In [81]:
client.vector_stores.files.upload_and_poll(        # Upload file
    vector_store_id=vector_store.id,
    file=make_file_object('dummy_3.txt', "Wine is made from grapes")   
)

VectorStoreFile(id='file-VKuiNbt7GXgwM3jJa6Z6BM', created_at=1756123956, last_error=None, object='vector_store.file', status='completed', usage_bytes=1048, vector_store_id='vs_68ac4cff5f1c81919b3bfda45ea803bb', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [23]:
client.files.retrieve("file-63688cr2RmynfybwyM78vG")

FileObject(id='file-63688cr2RmynfybwyM78vG', bytes=22, created_at=1756066404, filename='dummy_2.txt', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)

In [25]:
results = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="How is wine made?",
)

In [65]:
??make_file_object

Signature: make_file_object(name, content)
Docstring: <no docstring>
Source:   
def make_file_object(name,content):
    return (name, io.BytesIO(content.encode('utf-8')), 'text/plain')
File:      /tmp/ipykernel_709/3135118408.py
Type:      function

In [83]:
add_file_object(make_file_object('xxx','lorem ipsum'))

In [ ]:
add_file_object(make_file_object('xxx',df.head()['bodyContent'].values[0]))

BadRequestError: Error code: 400 - {'error': {'message': 'File type not supported', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [85]:
df.head(10).apply(lambda row: make_file_object(row['webUrl'], row['bodyContent']), axis=1).apply(add_file_object)